# Classification with Logistic Regression

Like every other person new to machine learning, I thought I'd try Andrew Ng's classic course on deep learning. So here's an article on logistic regression, which takes the first two weeks of the course.

## Model

Assume that you have a collection of images